<a href="https://colab.research.google.com/github/itsmepriyabrata/priyabrata_ai_python/blob/main/Graph_algorithm_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Graph attention networks

In [3]:
import tensorflow as tf

class GATConv(tf.keras.layers.Layer):
  """
  A single Graph Attention Network (GAT) layer implementation in TensorFlow.
  """
  def __init__(self, in_features, out_features, num_heads=1, concat=True, **kwargs):
    super(GATConv, self).__init__(**kwargs)
    self.in_features = in_features
    self.out_features = out_features
    self.num_heads = num_heads
    self.concat = concat

    self.W = tf.keras.layers.Dense(out_features * num_heads, use_bias=False)

    self.a = tf.keras.layers.Dense(1, use_bias=True)

    self.leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)

  def call(self, inputs, training=None):
    """
    Forward pass of the GAT layer.

    Args:
        inputs: A tuple of tensors representing (node features, adjacency matrix).
        training: Boolean indicating training mode (optional).

    Returns:
        A tensor representing the output features of the GAT layer.
    """
    node_features, adj_matrix = inputs

    Wh = tf.split(self.W(node_features), self.num_heads, axis=-1)

    if adj_matrix.shape[-1] != Wh[0].shape[-1]:
      adj_matrix = tf.reshape(adj_matrix, (-1, 1, adj_matrix.shape[-1]))  # Add a dimension
      adj_matrix = tf.cast(adj_matrix, dtype=Wh[0].dtype)  # Cast to match Wh's data type

    a_input = tf.concat([tf.expand_dims(Wh[i], axis=1) for i in range(self.num_heads)], axis=-1)
    a_input = tf.concat([a_input, adj_matrix], axis=-1)
    e = self.leaky_relu(self.a(a_input))

    attention = tf.nn.softmax(e, axis=-2)

    output = tf.matmul(attention, tf.transpose(Wh[0], perm=[0, 2, 1]))
    for i in range(1, self.num_heads):
      output = tf.concat([output, tf.matmul(attention, tf.transpose(Wh[i], perm=[0, 2, 1]))], axis=-1)

    if self.concat:
      output = tf.concat(output, axis=-1)

    return output

node_features = tf.random.uniform((10, 5))  # Replace with your actual data
adj_matrix = tf.ones((10, 10))  # Replace with your actual adjacency matrix (ensure compatible shape)

try:
  gat_layer = GATConv(5, 10, num_heads=2)
  output = gat_layer((node_features, adj_matrix))
  print(output.shape)
except Exception as e:
  print(f"An error occurred: {e}")


An error occurred: Exception encountered when calling layer 'gat_conv_2' (type GATConv).

{{function_node __wrapped__ConcatV2_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Ranks of all input tensors should match: shape[0] = [10,1,20] vs. shape[1] = [10,10] [Op:ConcatV2] name: concat

Call arguments received by layer 'gat_conv_2' (type GATConv):
  • inputs=('tf.Tensor(shape=(10, 5), dtype=float32)', 'tf.Tensor(shape=(10, 10), dtype=float32)')
  • training=None


GraphSAGE

In [4]:
import tensorflow as tf

class GraphSAGEConv(tf.keras.layers.Layer):
  """
  A GraphSAGE convolutional layer implementation in TensorFlow.
  """
  def __init__(self, in_features, out_features, aggregate_func="mean", bias=False, **kwargs):
    super(GraphSAGEConv, self).__init__(**kwargs)
    self.in_features = in_features
    self.out_features = out_features
    self.aggregate_func = aggregate_func
    self.bias = bias

    self.W = tf.keras.layers.Dense(out_features, use_bias=bias)

  def call(self, inputs, training=None):
    """
    Forward pass of the GraphSAGE layer.

    Args:
        inputs: A tuple of tensors representing (node features, neighbor features, sampling mask).
        training: Boolean indicating training mode (optional).

    Returns:
        A tensor representing the aggregated node features.
    """
    node_features, neighbor_features, mask = inputs

    self_transformed_features = self.W(node_features)

    if self.aggregate_func == "mean":
      aggregated_features = tf.math.reduce_mean(neighbor_features * mask, axis=1)
    elif self.aggregate_func == "sum":
      aggregated_features = tf.math.reduce_sum(neighbor_features * mask, axis=1)
    else:
      raise ValueError(f"Invalid aggregate function: {self.aggregate_func}")

    combined_features = tf.concat([self_transformed_features, aggregated_features], axis=-1)

    output = tf.nn.relu(combined_features)
    return output

class GraphSAGE(tf.keras.Model):
  """
  A basic GraphSAGE model with multiple convolutional layers.
  """
  def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, aggregate_func="mean", **kwargs):
    super(GraphSAGE, self).__init__(**kwargs)
    self.layers = []

    self.layers.append(GraphSAGEConv(input_dim, hidden_dim, aggregate_func=aggregate_func))

    for _ in range(1, num_layers):
      self.layers.append(GraphSAGEConv(hidden_dim, hidden_dim, aggregate_func=aggregate_func))

    self.layers.append(GraphSAGEConv(hidden_dim, output_dim, aggregate_func="mean"))

  def call(self, inputs, training=None):
    """
    Forward pass of the GraphSAGE model.

    Args:
        inputs: A tuple of tensors representing (node features, neighbor features list, sampling mask list).
        training: Boolean indicating training mode (optional).

    Returns:
        A tensor representing the node embeddings.
    """
    node_features = inputs[0]
    neighbor_features_list, sampling_mask_list = inputs[1:]

    hidden_features = node_features
    for layer, neighbor_features, sampling_mask in zip(self.layers, neighbor_features_list, sampling_mask_list):
      hidden_features = layer((hidden_features, neighbor_features, sampling_mask))

    return hidden_features

def sample_neighbors(adj_matrix, num_neighbors):
  neighbor_indices = tf.random.uniform((adj_matrix.shape[0], num_neighbors), minval=0, maxval=adj_matrix.shape[0], dtype=tf.int32)
  neighbor_features = tf.gather(adj_matrix, neighbor_indices, axis=-1)
  return neighbor_features

node_features = tf.random.uniform((10, 5))  # Replace with your actual data
adj_matrix = tf.ones((10, 10))  # Replace with your actual adjacency matrix



Graph isomorphism networks

In [7]:
import tensorflow as tf

class GINConv(tf.keras.layers.Layer):
  """
  A single Graph Isomorphism Network (GINConv) layer implementation in TensorFlow.
  """
  def __init__(self, in_features, out_features, **kwargs):
    super(GINConv, self).__init__(**kwargs)
    self.in_features = in_features
    self.out_features = out_features

    self.w1 = tf.keras.layers.Dense(out_features, use_bias=False)  # For message passing
    self.w2 = tf.keras.layers.Dense(out_features, use_bias=False)  # For updating node features

    self.act = tf.keras.activations.relu

  def call(self, inputs, training=None):
    """
    Forward pass of the GINConv layer.

    Args:
        inputs: A tuple of tensors representing (node features, adjacency matrix).
        training: Boolean indicating training mode (optional).

    Returns:
        A tensor representing the updated node features after message passing.
    """
    node_features, adj_matrix = inputs

    message = tf.matmul(adj_matrix, self.w1(node_features))
    message = self.act(message)

    output = self.w2(node_features) + message

    return output

node_features = tf.random.uniform((10, 5))
adj_matrix = tf.ones((10, 10))
gin_conv = GINConv(5, 10)
output = gin_conv((node_features, adj_matrix))
print(output.shape)

(10, 10)


Random Walk

In [8]:
import random

class Graph:
  """
  A simple graph data structure using an adjacency list representation.
  """
  def __init__(self):
    self.adj_list = {}

  def add_edge(self, source, destination):
    """
    Adds a directed edge from source node to destination node.
    """
    if source not in self.adj_list:
      self.adj_list[source] = []
    self.adj_list[source].append(destination)

  def random_walk(self, start_node, num_steps):
    """
    Performs a random walk on the graph starting from a given node
    and taking a specified number of steps.

    Args:
        start_node: The node to start the walk from.
        num_steps: The number of steps to take in the walk.

    Returns:
        A list of nodes visited during the random walk.
    """
    walk = [start_node]
    current_node = start_node
    for _ in range(num_steps):
      if current_node not in self.adj_list:
        break  # No neighbors, end the walk
      next_node = random.choice(self.adj_list[current_node])
      walk.append(next_node)
      current_node = next_node
    return walk

graph = Graph()
graph.add_edge("A", "B")
graph.add_edge("A", "C")
graph.add_edge("B", "D")
graph.add_edge("C", "E")

start_node = "A"
num_steps = 5

walk = graph.random_walk(start_node, num_steps)
print(f"Random walk from {start_node} for {num_steps} steps: {walk}")


Random walk from A for 5 steps: ['A', 'B', 'D']


community detection algorithms

In [10]:
!pip install --upgrade networkx

import networkx as nx

graph = nx.Graph()
graph.add_edge("A", "B")
graph.add_edge("A", "C")
graph.add_edge("B", "D")
graph.add_edge("C", "E")
graph.add_edge("D", "E")
graph.add_edge("E", "F")
graph.add_edge("B", "F")

def modularity(graph, communities):
  total_edges = graph.number_of_edges()
  mod = 0
  for community in communities:
    for i in community:
      for j in community:
        if graph.has_edge(i, j):
          mod += 1 / (2 * total_edges) - (graph.degree[i] * graph.degree[j]) / (2 * total_edges**2)
  return mod

communities = list(nx.community.greedy_modularity_communities(graph))

print("Detected communities:")
for i, community in enumerate(communities):
  print(f"Community {i+1}: {community}")

Detected communities:
Community 1: frozenset({'D', 'E', 'B', 'F'})
Community 2: frozenset({'A', 'C'})
